Presence = 1
Absence = 0

In [1]:
#librerías
import pandas as pd
import numpy as np

In [6]:
#leer el dataset de entrenamiento y mostrar su info general
train = pd.read_csv("data/train.csv")
train.head()

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence


In [ ]:
#info general del dataset
train.info()

<class 'pandas.DataFrame'>
RangeIndex: 630000 entries, 0 to 629999
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       630000 non-null  int64  
 1   Age                      630000 non-null  int64  
 2   Sex                      630000 non-null  int64  
 3   Chest pain type          630000 non-null  int64  
 4   BP                       630000 non-null  int64  
 5   Cholesterol              630000 non-null  int64  
 6   FBS over 120             630000 non-null  int64  
 7   EKG results              630000 non-null  int64  
 8   Max HR                   630000 non-null  int64  
 9   Exercise angina          630000 non-null  int64  
 10  ST depression            630000 non-null  float64
 11  Slope of ST              630000 non-null  int64  
 12  Number of vessels fluro  630000 non-null  int64  
 13  Thallium                 630000 non-null  int64  
 14  Heart Disease  

In [7]:
#ahora empecemos la limpieza, primero hagamos las columnas un poco más faciles de leer usando snake.case

train.columns = train.columns.str.strip().str.lower().str.replace(r"[-/\s+]", "_", regex=True)
train.columns

Index(['id', 'age', 'sex', 'chest_pain_type', 'bp', 'cholesterol',
       'fbs_over_120', 'ekg_results', 'max_hr', 'exercise_angina',
       'st_depression', 'slope_of_st', 'number_of_vessels_fluro', 'thallium',
       'heart_disease'],
      dtype='str')

In [8]:
#También podemos hacer que las columnas categoricas sean mas faciles de leer
train["heart_disease"] = train["heart_disease"].str.strip().str.lower().str.replace(r"[-/\s+]", "_", regex=True)
train.head()

,id,age,sex,chest_pain_type,bp,cholesterol,fbs_over_120,ekg_results,max_hr,exercise_angina,st_depression,slope_of_st,number_of_vessels_fluro,thallium,heart_disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,presence


In [ ]:
#veamos si hay valores duplicados, si los hay entonces podriamos eliminarlos
train.duplicated().value_counts()

False    630000
Name: count, dtype: int64

In [11]:
train.duplicated(["id"]).value_counts()


False    630000
Name: count, dtype: int64

In [12]:
train.duplicated(subset=train.columns.drop("id"), keep=False).value_counts()

False    630000
Name: count, dtype: int64

In [14]:
train["id"] = train["id"].astype("str")
train.info()

<class 'pandas.DataFrame'>
RangeIndex: 630000 entries, 0 to 629999
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       630000 non-null  str    
 1   age                      630000 non-null  int64  
 2   sex                      630000 non-null  int64  
 3   chest_pain_type          630000 non-null  int64  
 4   bp                       630000 non-null  int64  
 5   cholesterol              630000 non-null  int64  
 6   fbs_over_120             630000 non-null  int64  
 7   ekg_results              630000 non-null  int64  
 8   max_hr                   630000 non-null  int64  
 9   exercise_angina          630000 non-null  int64  
 10  st_depression            630000 non-null  float64
 11  slope_of_st              630000 non-null  int64  
 12  number_of_vessels_fluro  630000 non-null  int64  
 13  thallium                 630000 non-null  int64  
 14  heart_disease  